In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Flatten,Reshape
import os
import numpy as np
from pandas import DataFrame as df
import pandas as pd
from sklearn.datasets import fetch_openml
from tqdm import tqdm
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
from numpy.linalg import svd

pca = PCA()


x_train, y_train = fetch_openml('mnist_784', version=1, return_X_y=True)
x_train = x_train.reshape((70000,784))/255.0

In [ ]:
model = Sequential()
model.add(Flatten(input_shape=x_train[0].shape))
#model.add(Dense(784,activation='sigmoid',use_bias=False))
#model.add(Dense(16,activation='sigmoid',use_bias=False))
model.add(Dense(2,activation='sigmoid',use_bias=False))
#model.add(Dense(16,activation='sigmoid',use_bias=False))
model.add(Dense(784,activation='selu',use_bias=False))
#model.add(Reshape((28,28)))

In [ ]:
model.compile(loss='mae',optimizer='sgd')


In [ ]:
model.fit(x_train,x_train,epochs=10,batch_size=1000)

In [ ]:
weights = model.get_weights()

In [ ]:
ae_fin = np.dot(x_train,weights[0])

In [ ]:
pca_fin = pca.fit_transform(x_train)

In [ ]:
master = df(pca_fin[:,:3])
master.columns=['x','y','z']
master = master.join(pd.Series(y_train,name='labels'))
fig = plt.figure(figsize=(10,10))
for k in [221,222,223,224]:
    if k==224:
        ax = fig.add_subplot(k,projection='3d')
    else:
        ax = fig.add_subplot(k)
    for l in np.sort(master.labels.unique()):
        temp = master[master['labels']==l]
        x= temp['x'].values
        y = temp['y'].values
        z = temp['z'].values
        if k == 221:
            
            ax.scatter(x,y,s = .03,label = int(l))
            ax.set_xlabel('First autoencoded dimension')
            ax.set_ylabel('Second autoencoded dimension')
        elif k==222:
            ax.scatter(x,z,s=.03,label=int(l))
            ax.set_xlabel('First autoencoded dimension')
            ax.set_ylabel('Third autoencoded dimension')
        elif k == 223:
            ax.scatter(y,z,s=.03,label=int(l))
            ax.set_xlabel('Second autoencoded dimension')
            ax.set_ylabel('Third autoencoded dimension')
        else:
            ax.scatter(x,y,z,s = .03,label=int(l))
            ax.set_xlabel('First autoencoded dimension')
            ax.set_ylabel('Second autoencoded dimension')
            ax.set_zlabel('Third autoencoded dimension')
    
    #plt.xlabel('First Autoencoded Dimension')
    
plt.legend(markerscale=50,loc=(1.1,.8))
#plt.set_size(20,20)
plt.savefig('sklearn_3_pca.png',dpi = 1000)

In [ ]:
master = df(ae_fin)
master.columns=['x','y']
master = master.join(pd.Series(y_train,name='labels'))
fig = plt.figure(figsize=(2.5,2.5))
ax = fig.add_subplot(111)
for l in np.sort(master.labels.unique()):
    temp = master[master['labels']==l]
    x= temp['x'].values
    y = temp['y'].values
    ax.scatter(x,y,s=.01,label=int(l))
    ax.set_xlabel('First autoencoded dimension')
    ax.set_ylabel('Second autoencoded dimension')
        
plt.legend(markerscale=50,loc=(1.1,-.1))
#plt.set_size(20,20)
#plt.savefig('sklearn_2_pca.png',dpi = 1000)

In [ ]:
U,E,T=np.linalg.svd(out,full_matrices = False)

In [ ]:
E.shape

In [ ]:
def relu(z):
    return(z if z>0 else 0)
relu = np.vectorize(relu)

def relu_der(z):
    return(1 if z>0 else 0.01)
relu_der = np.vectorize(relu_der)


def simple_logistic(z):
    return(1/(1+np.e**(-z)))

log = np.vectorize(simple_logistic)

def log_der(z):
    return(z*(1-z))

log_der = np.vectorize(log_der)

In [ ]:
def fb4(flat,w1,w2,w3,w4,lr):
    l1 = log(np.dot(flat,w1))
    l2 = log(np.dot(l1,w2))
    l3 = log(np.dot(l2,w3))
    l4 = log(np.dot(l3,w4))
    
    l4_err = flat-l4
    delt4 = l4_err*log_der(l4)
    w4_grads = l3.T.dot(delt4)

    l3_err = delt4.dot(w4.T)
    delt3 = l3_err*log_der(l3)
    w3_grads = l2.T.dot(delt3)

    l2_err = delt3.dot(w3.T)
    delt2 = l2_err*log_der(l2)
    w2_grads = l1.T.dot(delt2)

    l1_err = delt2.dot(w2.T)
    delt1 = l1_err*log_der(l1)
    w1_grads = flat.T.dot(delt1)

    w4 = w4+lr*w4_grads
    w3 = w3+lr*w3_grads
    w2 = w2+lr*w2_grads
    w1 = w1+lr*w1_grads
    
    print(np.mean(np.abs(l4_err)))
    return(w1,w2,w3,w4)

In [ ]:
def fb2(flat,w1,w2,lr):
    l1 = log(np.dot(flat,w1))
    l2 = log(np.dot(l1,w2))

    l2_err = flat-l2
    delt2 = l2_err*log_der(l2)
    w2_grads = l1.T.dot(delt2)

    l1_err = delt2.dot(w2.T)
    delt1 = l1_err*log_der(l1)
    w1_grads = flat.T.dot(delt1)

    w2 = w2+lr*w2_grads
    w1 = w1+lr*w1_grads
    
    print(np.mean(np.abs(l2_err)))
    return(w1,w2)

In [ ]:
pcs = 2
w1 = np.random.random((784,pcs))
w2 = np.random.random((pcs,784))

batch_size=1000

lr = .01
flat = x_train.copy()
for k in tqdm(range(10)):
    np.random.shuffle(flat)
    for b in range(int(len(flat)/batch_size)):
        temp_flat = flat[int(b*batch_size):int((b+1)*batch_size),:]
        w1,w2 = fb2(temp_flat,w1,w2,lr)
#np.save('weights_2_'+str(pcs)+'.npy',np.array([w1,w2]))

In [ ]:
out = log(x_train.dot(w1))

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.figure()
x,y = zip(*out)
plt.scatter(x,y)
plt.show()

In [ ]:
weights = []
for k in [1,2,3]:
    weights.append(np.load('weights_pca_'+str(k)+'.npy',allow_pickle=True))

In [ ]:
len(weights)

In [ ]:
x_train

In [ ]:
weights

In [ ]:
x = x_train.dot(weights[0][0])
y = x_train.dot(weights[1][0])

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.figure()
plt.scatter(x,y)
plt.show()

In [ ]:
from pandas import DataFrame as df
import pandas as pd

In [ ]:
out = [[x[k],y[k]] for k in range(len(x))]
master = df(out)
master.columns=['x','y']
master = master.join(pd.Series(y_train,name='labels'))
plt.figure()
for l in np.sort(master.labels.unique()):
    temp = master[master['labels']==l]
    x,y = zip(*temp[['x','y']].values)
    plt.scatter(x,y,s = .1,label = l)
plt.legend(markerscale=20)

In [ ]:
#for k in [1,2,3]:
pcs = 2
w1,w2,w3,w4 = np.load('weights_4_'+str(pcs)+'.npy',allow_pickle=True)
out = log(flat.dot(w1).dot(w2))
out.shape

In [ ]:
master = df(out)
master.columns=['x','y']
master = master.join(pd.Series(y_train,name='labels'))
plt.figure()
for l in np.sort(master.labels.unique()):
    temp = master[master['labels']==l]
    x,y = zip(*temp[['x','y']].values)
    plt.scatter(x,y,s = .1,label = l)
#plt.xscale('log')
#plt.yscale('log')
plt.legend(markerscale=20)

In [ ]:
lr = .001
batch_size=1000
new_flat = x_train.copy()
for k in [1]:
    w1 = np.random.random((784,k))
    w2 = np.random.random((k,784))
    
    for e in tqdm(range(10)):
        np.random.shuffle(new_flat)
        for b in range(int(len(new_flat)/batch_size)):
            temp_flat =  new_flat[int(b*batch_size):int((b+1)*batch_size),:]
            w1,w2 = fb2(temp_flat,w1,w2,lr)
    np.save('weights_pca_'+str(k)+'.npy',np.array([w1,w2]))
    new_flat = x_train-(log(log(x_train.dot(w1)).dot(w2)))

In [ ]:
pca_fin 

In [ ]:
x_train.shape

In [ ]:
U,E,Vh = svd(x_train, full_matrices = False,compute_uv=True)

In [ ]:
U[:,:2].dot(np.diag(E)[:2,:]).dot(Vh.T[:2,:])

In [ ]:
x_train.dot(Vh.T)

In [ ]:
pca_fin

In [ ]:
U.shape

In [ ]:
E.shape

In [ ]:
Vh.shape

In [ ]:
pca_fin.round(2)

In [ ]:
E_hat = np.zeros((70000,784))
for p in range(784):
    E_hat[p][p] = E[p]


In [ ]:
E_hat

In [ ]:
np.dot(x_train,E_hat.T)

In [ ]:
pca_fin.round(2)